# Spaceship KNN Submission

Hello there whoever handles this aku lupa, ini cuma coba-coba aja si pake KNeighborsClassifier buat test submission. Feel free to modify ya, lov u makasi semangat. -Egar

In [ ]:
!pip install catboost==1.2.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 10.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, scaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier
from sklearn.preprocessing import MinMaxScaler

ImportError: cannot import name 'scaler' from 'sklearn.preprocessing' (/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/preprocessing/__init__.py)

# Bring the Data In

In [ ]:
# Load data
X_df = pd.read_csv('preprocessedbankchurn_train.csv')
X_df.drop("Exited", axis=1, inplace=True)
y_df = pd.read_csv('preprocessedbankchurn_train.csv')['Exited']
X_submission = pd.read_csv('preprocessedbankchurn_test.csv')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [ ]:
model = CatBoostClassifier(random_state = 42, verbose=0)
models = model.fit(X_train, y_train)
importance = models.feature_importances_
tabel = X_df.columns
for f, fi in zip(tabel, importance):
    print('Feature : %24s, score %.5f' %(f, fi))

Feature :                       id, score 3.94387
Feature :               CustomerId, score 4.24935
Feature :              CreditScore, score 4.98867
Feature :                      Age, score 16.64506
Feature :                   Tenure, score 2.17648
Feature :                  Balance, score 7.77768
Feature :            NumOfProducts, score 27.76303
Feature :                HasCrCard, score 0.81067
Feature :           IsActiveMember, score 6.87257
Feature :          EstimatedSalary, score 4.55135
Feature :                   France, score 0.24190
Feature :                  Germany, score 3.92398
Feature :                    Spain, score 0.16059
Feature :                   Female, score 1.39000
Feature :                     Male, score 1.21864
Feature :         ExitedfamilySize, score 6.81943
Feature :      NotExitedfamilySize, score 6.46674


In [ ]:
X1 = X_df.drop(columns=['Female','Male','Germany','Spain','France','HasCrCard','Tenure']).values
X_train, X_test, y_train, y_test = train_test_split(X1, y_df, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(132027, 10)
(33007, 10)
(132027,)
(33007,)


# Model Training

In [ ]:
# Membuat pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('classifier', CatBoostClassifier(random_seed=42, verbose=0))
])


param_grid = {
    'classifier__iterations': [1000],
    'classifier__learning_rate': [0.01],
    'classifier__depth': [10],
    'classifier__l2_leaf_reg': [10],
    'classifier__border_count': [128],
    'classifier__loss_function': ['CrossEntropy']
}


# Menggunakan StratifiedKFold untuk handling imbalanced data
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
# GridSearchCV setup
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='roc_auc', verbose=1)
grid_search.fit(X_train, y_train)

# Evaluate the model
best_model = grid_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
print("Best parameters:", grid_search.best_params_)
print("ROC-AUC on test set:", roc_auc_score(y_test, y_pred_proba))

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best parameters: {'classifier__border_count': 128, 'classifier__depth': 10, 'classifier__iterations': 1000, 'classifier__l2_leaf_reg': 10, 'classifier__learning_rate': 0.01, 'classifier__loss_function': 'CrossEntropy'}
ROC-AUC on test set: 0.8883030046747185


In [ ]:
# Evaluation on the test set
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
grid_search.fit(X1, y_df)

              precision    recall  f1-score   support

           0       0.88      0.95      0.92     26052
           1       0.74      0.54      0.62      6955

    accuracy                           0.86     33007
   macro avg       0.81      0.74      0.77     33007
weighted avg       0.86      0.86      0.85     33007

Confusion Matrix:
[[24758  1294]
 [ 3218  3737]]
Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('classifier',
                                        <catboost.core.CatBoostClassifier object at 0x7f68bf033c40>)]),
             param_grid={'classifier__border_count': [128],
                         'classifier__depth': [10],
                         'classifier__iterations': [1000],
                         'classifier__l2_leaf_reg': [10],
                         'classifier__learning_rate': [0.01],
                         'classifier__loss_function': ['CrossEntropy']},
             scoring='roc_auc', verbose=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Inisialisasi scaler
scaler = MinMaxScaler()
# Fit scaler pada data dengan 10 fitur
scaler.fit(X_train)  # Asumsi X_train_10_features adalah data yang sudah di-drop kolom

# Gunakan scaler yang sudah di-fit untuk transform data lain yang memiliki 10 fitur yang sama
X1_scaled = scaler.transform(X1)
X_submission_10_features = X_submission.drop(columns=['Female','Male','Germany','Spain','France','HasCrCard','Tenure'])

# Transform X_submission dengan scaler yang sudah di-fit
X_submission_scaled = scaler.transform(X_submission_10_features)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [ ]:
# Memeriksa jumlah fitur pada setiap dataset
print("Jumlah fitur pada X_train_:", X_train.shape[1])
print("Jumlah fitur pada X1:", X1.shape[1])
print("Jumlah fitur pada X_submission_10_features:", X_submission_10_features.shape[1])


Jumlah fitur pada X_train_: 10
Jumlah fitur pada X1: 10
Jumlah fitur pada X_submission_10_features: 10


In [ ]:
# Predict probabilities for submission data
y_submission_proba = grid_search.predict_proba(X_submission_10_features)[:, 1]  # Probabilities of Exited
print(y_submission_proba)

# Prepare submission dictionary
submission_dict = {'id': X_submission_10_features['id'], 'Exited': y_submission_proba}
submission_dict = pd.DataFrame(submission_dict)
submission_dict


/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
[0.00711081 0.80304904 0.01386681 ... 0.02746288 0.13298822 0.17573997]


,id,Exited
0,165034,0.007111
1,165035,0.803049
2,165036,0.013867
3,165037,0.274742
4,165038,0.364523
...,...,...
110018,275052,0.080282
110019,275053,0.086879
110020,275054,0.027463
110021,275055,0.132988


In [ ]:
submission_dict.to_csv('bankChurn_cat3.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2a79941c-6614-47fe-9427-0e9f23998893' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>